## Task 1: Sentiment Labelling

In [ ]:
import pandas as pd
from textblob import TextBlob

# STEP 1: Load the dataset
df = pd.read_csv('test.csv')

# STEP 2: Clean column names
df.columns = df.columns.str.strip()

# STEP 3: Check and use the correct column for message body
if 'body' not in df.columns:
    raise KeyError("Expected 'body' column not found.")

# STEP 4: Define sentiment classification function
def get_sentiment(text):
    polarity = TextBlob(str(text)).sentiment.polarity
    if polarity >= 0.05:
        return 'Positive'
    elif polarity <= -0.05:
        return 'Negative'
    else:
        return 'Neutral'

# STEP 5: Apply sentiment function to 'body' column
df['Sentiment'] = df['body'].apply(get_sentiment)

# STEP 6: Save or preview results
print(df[['body', 'Sentiment']].head())  # Show first few rows
df.to_csv('test_with_sentiment.csv', index=False)  # Save with sentiment


## Task 2: Exploratory Data Analysis (EDA)

In [ ]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Create visualizations folder if it doesn't exist
os.makedirs('visualizations', exist_ok=True)

# Load sentiment-labeled dataset
df = pd.read_csv('test_with_sentiment.csv')
df.columns = df.columns.str.strip()
df['date'] = pd.to_datetime(df['date'])
df['month'] = df['date'].dt.to_period('M')

# Plot sentiment distribution
plt.figure(figsize=(6,4))
sns.countplot(x='Sentiment', data=df, palette='Set2', legend=False)
plt.title("Sentiment Distribution")
plt.xlabel("Sentiment")
plt.ylabel("Count")
plt.tight_layout()
plt.savefig('visualizations/sentiment_distribution.png')  # Now it will work
plt.show()


## Task 3: Monthly Sentiment Score Calculation

In [ ]:
import pandas as pd

# Load sentiment-labeled data
df = pd.read_csv('test_with_sentiment.csv')

# Clean column names and convert date
df.columns = df.columns.str.strip()
df['date'] = pd.to_datetime(df['date'])
df['month'] = df['date'].dt.to_period('M')

# Step 1: Assign score values to sentiments
sentiment_scores = {
    'Positive': 1,
    'Neutral': 0,
    'Negative': -1
}
df['Score'] = df['Sentiment'].map(sentiment_scores)

# Step 2: Group by 'from' (employee ID) and 'month' and sum the scores
monthly_scores = df.groupby(['from', 'month'])['Score'].sum().reset_index()
monthly_scores = monthly_scores.rename(columns={'from': 'Employee_ID', 'Score': 'Monthly_Sentiment_Score'})

# Step 3: Save the result
monthly_scores.to_csv('monthly_sentiment_scores.csv', index=False)
print(monthly_scores.head())


## Task 4: Employee Ranking

In [ ]:
import pandas as pd

# Load the monthly sentiment score data
df = pd.read_csv('monthly_sentiment_scores.csv')

# Ensure correct types
df['month'] = pd.PeriodIndex(df['month'], freq='M')

# Function to get top 3 positive and negative per month
def rank_employees(group):
    top_pos = group.sort_values(by=['Monthly_Sentiment_Score', 'Employee_ID'], ascending=[False, True]).head(3)
    top_neg = group.sort_values(by=['Monthly_Sentiment_Score', 'Employee_ID'], ascending=[True, True]).head(3)
    return top_pos, top_neg

# Prepare containers for results
top_positive_list = []
top_negative_list = []

# Loop through each month and rank employees
for month, group in df.groupby('month'):
    top_pos, top_neg = rank_employees(group)
    top_pos['Rank_Type'] = 'Top Positive'
    top_neg['Rank_Type'] = 'Top Negative'
    top_positive_list.append(top_pos)
    top_negative_list.append(top_neg)

# Concatenate all rankings
top_positives = pd.concat(top_positive_list)
top_negatives = pd.concat(top_negative_list)

# Combine both for reporting
all_rankings = pd.concat([top_positives, top_negatives])
all_rankings = all_rankings.sort_values(by=['month', 'Rank_Type', 'Monthly_Sentiment_Score'], ascending=[True, True, False])

# Save to CSV
all_rankings.to_csv('employee_monthly_rankings.csv', index=False)

# Display example output
print(all_rankings.head())


## Task 5: Flight Risk Detection

In [ ]:
import pandas as pd

# Load data
df = pd.read_csv('test_with_sentiment.csv')
df.columns = df.columns.str.strip()
df['date'] = pd.to_datetime(df['date'])

# Filter only negative messages
negative_msgs = df[df['Sentiment'] == 'Negative'].copy()
negative_msgs = negative_msgs.sort_values(by=['from', 'date'])

# Function to find if there are 4 or more negative messages in any 30-day rolling window
def find_flight_risks(group):
    group = group.sort_values('date').reset_index(drop=True)
    dates = group['date']
    for i in range(len(dates)):
        count = 1
        for j in range(i + 1, len(dates)):
            if (dates[j] - dates[i]).days <= 30:
                count += 1
                if count >= 4:
                    return pd.Series({'Employee_ID': group['from'].iloc[0]})
            else:
                break
    return None

# Apply the check to each employee group
flight_risk_employees = negative_msgs.groupby('from').apply(find_flight_risks).dropna().reset_index(drop=True)

# Save output
flight_risk_employees.to_csv('flight_risk_employees.csv', index=False)
print("Flight Risk Employees:\n", flight_risk_employees)


## Task 6: Predictive Modeling

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score

# Load and preprocess
df = pd.read_csv('test_with_sentiment.csv')
df.columns = df.columns.str.strip()
df['date'] = pd.to_datetime(df['date'])
df['month'] = df['date'].dt.to_period('M')

# Feature engineering
df['message_length'] = df['body'].astype(str).apply(len)
df['word_count'] = df['body'].astype(str).apply(lambda x: len(x.split()))
df['score'] = df['Sentiment'].map({'Positive': 1, 'Neutral': 0, 'Negative': -1})

# Monthly aggregation
agg_df = df.groupby(['from', 'month']).agg({
    'message_length': 'mean',
    'word_count': 'mean',
    'body': 'count',
    'score': 'sum'
}).reset_index()

agg_df = agg_df.rename(columns={
    'from': 'Employee_ID',
    'body': 'message_count',
    'score': 'monthly_sentiment_score'
})

# Train-test split
X = agg_df[['message_length', 'word_count', 'message_count']]
y = agg_df['monthly_sentiment_score']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Linear Regression
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Evaluation
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error (MAE):", round(mae, 2))
print("R-squared (R²):", round(r2, 2))

# 📊 Plot 1: Actual vs Predicted
plt.figure(figsize=(6, 4))
plt.scatter(y_test, y_pred, color='blue', alpha=0.6)
plt.plot([y.min(), y.max()], [y.min(), y.max()], '--', color='red')
plt.xlabel('Actual Sentiment Score')
plt.ylabel('Predicted Sentiment Score')
plt.title('Actual vs Predicted Sentiment Score')
plt.tight_layout()
plt.savefig('visualizations/actual_vs_predicted.png')
plt.show()

# 📊 Plot 2: Residual Plot
residuals = y_test - y_pred
plt.figure(figsize=(6, 4))
plt.scatter(y_pred, residuals, alpha=0.6)
plt.axhline(y=0, color='red', linestyle='--')
plt.xlabel('Predicted Sentiment Score')
plt.ylabel('Residuals (Actual - Predicted)')
plt.title('Residual Plot')
plt.tight_layout()
plt.savefig('visualizations/residual_plot.png')
plt.show()
